In [1]:
# from pydap.client import open_url
import os
from pydap.client import open_url
import xarray as xr
import datetime
from datetime import timedelta, date
import netCDF4 as nc
from netCDF4 import Dataset
import h5py
from net_cdf_functions import *
from degree_day_equations import single_sine_horizontal_cutoff
import numpy as np
%matplotlib inline
#import dask
from dask.distributed import Client

#check if port is already in use

#client = Client(n_workers=3, threads_per_worker=1, memory_limit="2GB")
#client

data_path = '/media/thom/Q/data/'
#data_path = 'data/test/'



ModuleNotFoundError: No module named 'numpy.lib.arrayterator'

In [13]:
client = Client(n_workers=3, threads_per_worker=1, memory_limit="2GB")
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 3
Total threads: 3,Total memory: 5.59 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40267,Workers: 3
Dashboard: http://127.0.0.1:8787/status,Total threads: 3
Started: Just now,Total memory: 5.59 GiB
Comm: tcp://127.0.0.1:41959,Total threads: 1
Dashboard: http://127.0.0.1:39835/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:46199,


In [91]:

# download data for eyars 2000 for january first
"""
for year in range(2020, 2021):
    for day in range(1, 2):
        fetch_and_save_data(year, 2, 10, data_path)
"""

'\nfor year in range(2020, 2021):\n    for day in range(1, 2):\n        fetch_and_save_data(year, 2, 10, data_path)\n'

In [9]:
# fetch and save data 2000-2020
for year in range(2019, 2020):
    fetch_and_save_data(year, 1, 10, data_path)

Saved data for 20190101 to /media/thom/Q/data/PRISM/2019/PRISM_combo_20190101.nc
Saved data for 20190102 to /media/thom/Q/data/PRISM/2019/PRISM_combo_20190102.nc
Saved data for 20190103 to /media/thom/Q/data/PRISM/2019/PRISM_combo_20190103.nc
Saved data for 20190104 to /media/thom/Q/data/PRISM/2019/PRISM_combo_20190104.nc
Saved data for 20190105 to /media/thom/Q/data/PRISM/2019/PRISM_combo_20190105.nc
Saved data for 20190106 to /media/thom/Q/data/PRISM/2019/PRISM_combo_20190106.nc
Saved data for 20190107 to /media/thom/Q/data/PRISM/2019/PRISM_combo_20190107.nc
Saved data for 20190108 to /media/thom/Q/data/PRISM/2019/PRISM_combo_20190108.nc
Saved data for 20190109 to /media/thom/Q/data/PRISM/2019/PRISM_combo_20190109.nc
Saved data for 20190110 to /media/thom/Q/data/PRISM/2019/PRISM_combo_20190110.nc


In [14]:
import xarray as xr


def subset_dataset_by_coords(dataset, lat, lon, window_size=None):
    # if window_size is not None:

    if window_size is not None:
        # Define the window boundaries
        lat_min = lat - window_size
        lat_max = lat + window_size
        lon_min = lon - window_size
        lon_max = lon + window_size

        # Create boolean masks for the coordinate ranges
        mask_lat = (dataset.latitude >= lat_min) & (dataset.latitude <= lat_max)
        mask_lon = (dataset.longitude >= lon_min) & (dataset.longitude <= lon_max)

        # Create a boolean mask that combines latitude and longitude masks
        # ombined_mask = mask_lat[:, np.newaxis] & mask_lon[np.newaxis, :]

        # Apply the mask using .where() and drop the data points outside the window
        subset = dataset.where(mask_lat & mask_lon, drop=True).fillna(np.nan)

        return subset
    else:
        return dataset.sel(latitude=lat, longitude=lon, method="nearest")

In [15]:
prism_year_stack = xr.open_mfdataset(
    data_path + "PRISM/*/PRISM_combo_*.nc",
    # combine="by_coords",
    drop_variables=["ppt", "tmean"], 
)
# prism_year_stack = prism_year_stack.chunk({"latitude": 100, "longitude": 100})

la_combined = subset_dataset_by_coords(
    prism_year_stack, 34.05, -118.25, window_size=None
)
la_window = subset_dataset_by_coords(prism_year_stack, 33.896252, -118.408213, 0.25)
vsingle_sine_horizontal_cutoff = np.vectorize(single_sine_horizontal_cutoff)

In [16]:
tmin = la_window["tmin"].values
tmax = la_window["tmax"].values

Exception ignored in: <function CachingFileManager.__del__ at 0x70aadabeaca0>
Traceback (most recent call last):
  File "/home/thom/miniforge3/envs/fruitflypheno/lib/python3.12/site-packages/xarray/backends/file_manager.py", line 250, in __del__
    self.close(needs_lock=False)
  File "/home/thom/miniforge3/envs/fruitflypheno/lib/python3.12/site-packages/xarray/backends/file_manager.py", line 234, in close
    file.close()
  File "src/netCDF4/_netCDF4.pyx", line 2618, in netCDF4._netCDF4.Dataset.close
  File "src/netCDF4/_netCDF4.pyx", line 2585, in netCDF4._netCDF4.Dataset._close
  File "src/netCDF4/_netCDF4.pyx", line 2113, in netCDF4._netCDF4._ensure_nc_success
RuntimeError: NetCDF: HDF error
malloc(): invalid size (unsorted)
Exception ignored in: <function CachingFileManager.__del__ at 0x7abc87d34b80>
Traceback (most recent call last):
  File "/home/thom/miniforge3/envs/fruitflypheno/lib/python3.12/site-packages/xarray/backends/file_manager.py", line 250, in __del__
Exception ignor

2024-07-29 12:08:40,251 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:41959' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'original-open_dataset-tmin-cb1f1b452c134e827d32425204124901', ('where-7d9ea49214c84b078dcc5f4925e5c26d', 118, 0, 0), ('where-7d9ea49214c84b078dcc5f4925e5c26d', 103, 0, 0), ('where-7d9ea49214c84b078dcc5f4925e5c26d', 129, 0, 0), ('where-7d9ea49214c84b078dcc5f4925e5c26d', 141, 0, 0), ('where-7d9ea49214c84b078dcc5f4925e5c26d', 100, 0, 0), ('where-7d9ea49214c84b078dcc5f4925e5c26d', 123, 0, 0), ('where-7d9ea49214c84b078dcc5f4925e5c26d', 107, 0, 0), ('where-7d9ea49214c84b078dcc5f4925e5c26d', 138, 0, 0), ('where-7d9ea49214c84b078dcc5f4925e5c26d', 139, 0, 0), ('where-7d9ea49214c84b078dcc5f4925e5c26d', 10, 0, 0), ('where-7d9ea49214c84b078dcc5f4925e5c26d', 102, 0, 0), ('where-7d9ea49214c84b078dcc5f4925e5c26d', 108, 0, 0), ('where-7d9ea49214c84b078dcc5f4925e5c26d', 112, 0, 0), ('where-7d9ea49214c84b078dcc5f492

OSError: [Errno -101] NetCDF: HDF error: '/media/thom/Q/data/PRISM/2005/PRISM_combo_20050107.nc'

In [6]:
# @dask.delayed
def da_calculate_degree_days(LTT, UTT, data):
    # returns a data array with the degree days

    degree_days_vec = xr.apply_ufunc(
        vsingle_sine_horizontal_cutoff,
        tmin,
        tmax,
        kwargs={"LTT": LTT, "UTT": UTT},
    )
    degree_days_da = xr.DataArray(degree_days_vec, coords=data["tmax"].coords)
    data = data.assign(degree_days=degree_days_da)
    data = (data.drop_vars(["tmax", "tmin"]),)
    dask = "paralellized"
    return data

In [7]:
la_combined = da_calculate_degree_days(12, 36, la_window)

/home/thom/miniforge3/envs/fruitflypheno/lib/python3.12/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: invalid value encountered in func (vectorized)
  outputs = ufunc(*inputs)


In [13]:
la_combined

<xarray.Dataset> Size: 3kB
Dimensions:      (t: 199)
Coordinates:
  * t            (t) datetime64[ns] 2kB 2000-01-02 2000-01-03 ... 2019-01-10
    longitude    float64 8B -118.2
    latitude     float64 8B 34.04
Data variables:
    degree_days  (t) float64 2kB 0.3629 1.17 2.345 3.199 ... 1.463 4.158 2.945

In [8]:
degree_days_vec = xr.apply_ufunc(
    vsingle_sine_horizontal_cutoff,
    la_window["tmin"].values,
    la_window["tmax"].values,
    kwargs={"LTT": 12, "UTT": 30},
    # vectorize=True,  # Ensure function is applied element-wise
    input_core_dims=[
        ["t", "longitude", "latitude"],
        ["t", "longitude", "latitude"],
    ],  # Adjust to your dimensions
    output_core_dims=[["t", "longitude", "latitude"]],  # Adjust to your dimensions
    dask="allowed",  # Use Dask if your dataset is large
)
degree_days_da = xr.DataArray(degree_days_vec, coords=la_window["tmax"].coords)
la_window_DD = la_window.assign(degree_days=degree_days_da)
la_window_DD = la_window_DD.drop_vars(["tmax", "tmin"])

la_window_DD

<xarray.Dataset> Size: 231kB
Dimensions:      (t: 199, longitude: 12, latitude: 12)
Coordinates:
  * t            (t) datetime64[ns] 2kB 2000-01-02 2000-01-03 ... 2019-01-10
  * longitude    (longitude) float64 96B -118.6 -118.6 -118.5 ... -118.2 -118.2
  * latitude     (latitude) float64 96B 34.12 34.08 34.04 ... 33.75 33.71 33.67
Data variables:
    degree_days  (t, latitude, longitude) float64 229kB 0.122 0.1148 ... nan nan

In [12]:
la_window_DD["degree_days"]

<xarray.DataArray 'degree_days' (t: 199, latitude: 12, longitude: 12)> Size: 229kB
array([[[1.22027957e-01, 1.14805494e-01, 1.00034895e-01, ...,
         1.21768305e-01, 3.78800560e-02, 3.06118928e-03],
        [8.60698810e-04, 1.05426681e-01, 1.17339043e-01, ...,
         1.39724515e-01, 1.36815464e-01, 1.15038911e-01],
        [3.76387541e-01, 4.73156453e-01, 5.30531341e-01, ...,
         3.62905754e-01, 3.07839645e-01, 2.32590620e-01],
        ...,
        [           nan,            nan,            nan, ...,
         1.29168482e+00, 1.15410345e+00, 1.16155986e+00],
        [           nan,            nan,            nan, ...,
         1.37977913e+00, 1.36966176e+00,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan]],

       [[5.03288849e-01, 4.51073496e-01, 4.77692870e-01, ...,
         1.08726428e+00, 1.01480449e+00, 1.03483477e+00],
        [6.08553583e-01, 5.89696659e-01, 7.02011209e-01, ...,
         1.04500405e+00, 1.10744247e+00, 1.16801684e+00],
        [1.05830335e+00, 1.08398620e+00, 1.12002616e+00, ...,
         1.17049231e+00, 1.25672830e+00, 1.25889685e+00],
...
        [           nan,            nan,            nan, ...,
         4.12038980e+00, 4.28920679e+00, 4.40257136e+00],
        [           nan,            nan,            nan, ...,
         3.70997087e+00, 4.10169517e+00,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan]],

       [[2.97596261e+00, 3.22237164e+00, 3.06089809e+00, ...,
         3.20527273e+00, 3.33216972e+00, 3.47053652e+00],
        [2.05520367e+00, 3.31668348e+00, 2.89174141e+00, ...,
         3.15581055e+00, 3.24088820e+00, 3.42580464e+00],
        [3.00138351e+00, 2.83534985e+00, 2.72069415e+00, ...,
         2.94521198e+00, 3.20265075e+00, 3.37500270e+00],
        ...,
        [           nan,            nan,            nan, ...,
         2.37090114e+00, 2.44199803e+00, 2.62799341e+00],
        [           nan,            nan,            nan, ...,
         2.59012125e+00, 2.24093455e+00,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan]]])
Coordinates:
  * t          (t) datetime64[ns] 2kB 2000-01-02 2000-01-03 ... 2019-01-10
  * longitude  (longitude) float64 96B -118.6 -118.6 -118.5 ... -118.2 -118.2
  * latitude   (latitude) float64 96B 34.12 34.08 34.04 ... 33.75 33.71 33.67

In [182]:
degree_days_vec

<xarray.DataArray (t: 152, longitude: 12, latitude: 12)> Size: 175kB
array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 0, 0]],

       [[0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        ...,
...
        ...,
        [3, 3, 3, ..., 2, 2, 0],
        [3, 3, 3, ..., 2, 1, 0],
        [3, 3, 3, ..., 1, 0, 0]],

       [[6, 5, 4, ..., 0, 0, 0],
        [6, 6, 4, ..., 0, 0, 0],
        [6, 6, 4, ..., 0, 0, 0],
        ...,
        [4, 4, 4, ..., 3, 3, 0],
        [5, 4, 4, ..., 3, 3, 0],
        [5, 5, 4, ..., 3, 0, 0]],

       [[4, 4, 3, ..., 0, 0, 0],
        [4, 4, 2, ..., 0, 0, 0],
        [4, 4, 2, ..., 0, 0, 0],
        ...,
        [3, 3, 3, ..., 2, 1, 0],
        [3, 3, 3, ..., 2, 2, 0],
        [3, 3, 3, ..., 2, 0, 0]]])
Coordinates:
  * t          (t) datetime64[ns] 1kB 2000-01-01 2000-01-02 ... 2020-01-05
  * longitude  (longitude) float64 96B -118.6 -118.6 -118.5 ... -118.2 -118.2
  * latitude   (latitude) float64 96B 34.12 34.08 34.04 ... 33.75 33.71 33.67

In [ ]:
def day_cumsum_reaches_threshold(degree_days, start_day, threshold):
    cumsum = degree_days['degree_days'].sel(t=slice(start_day, None)).cumsum()
    # date = cumsum.where(cumsum >= threshold, drop=True)[0]['t']
    date = cumsum.where(cumsum >= threshold, drop=True)[0]["t"].values.astype(
        "datetime64[D]"
    )

    return date

In [ ]:
#create a list of start dates for each year from 2000 to 2008
start_dates = []
model_month = 1
model_day = 1
for year in range(2000, 2009):
    start_dates.append(datetime.datetime(year, model_month, model_day))

In [ ]:
threshold = 15

# Compute the day when cumulative sum reaches the threshold starting from day 1
start_day = "2001-01-01"
day_reach_threshold_from_start = day_cumsum_reaches_threshold(
    la_combined_DD, start_day, threshold
)


In [ ]:
# la_combined_DD["degree_days"].isel(t=slice(4, None))
# repeat but use .sel for date 01-03-2001
la_combined_DD["degree_days"].sel(t=slice("2001-01-03", None))

In [ ]:
tmax[42]

In [ ]:
tmin[42]

In [ ]:
import xarray as xr
import numpy as np


def calculate_degree_days(tmax, tmin, base_temp=10):
    """
    Calculate degree days from tmax and tmin.
    Degree days = (tmax + tmin) / 2 - base_temp
    """
    mean_temp = (tmax + tmin) / 2
    degree_days = mean_temp - base_temp
    return degree_days


def process_dataset(dataset):
    """
    Apply degree days calculation and drop tmax and tmin variables.
    """
    # Directly access tmax and tmin
    tmax = dataset["tmax"]
    tmin = dataset["tmin"]

    # Print dimensions and coordinates for debugging
    print("Dimensions of tmax:", tmax.dims)
    print("Dimensions of tmin:", tmin.dims)
    print("Coordinates of dataset:", dataset.coords)

    # Calculate degree days using apply_ufunc
    degree_days = xr.apply_ufunc(
        single_sine_horizontal_cutoff,
        tmax,
        tmin,
        input_core_dims=[["t"], ["t"]],
        output_core_dims=[["t"]],  # No core dimensions for the output
        vectorize=True,
        dask="allowed",  # Ensure compatibility with dask
        output_dtypes=[float],  # Specify output data type
    )

    # Print dimensions of degree_days for debugging
    print("Dimensions of degree_days:", degree_days.dims)

    # Create a new DataArray with the degree days
    degree_days_da = xr.DataArray(
        degree_days,
        dims=["time"],  # Use 'time' dimension for the DataArray
        coords={
            "time": dataset["time"].values
        },  # Use 'time' coordinate from the original dataset
        name="degree_days",
    )

    # Add degree days to the dataset and drop tmax and tmin
    dataset = dataset.assign(degree_days=degree_days_da)
    dataset = dataset.drop_vars(["tmax", "tmin"])

    return dataset


# Example usage
base_dir = data_path
start_year = 2000
end_year = 2020
start_day = 1
end_day = 10

combined_dataset = load_and_concatenate_datasets(
    start_year, end_year, start_day, end_day, base_dir
)

# Process the dataset
processed_dataset = process_dataset(combined_dataset)

# Persist or compute if needed
processed_dataset = processed_dataset.persist()

# Subset dataset by coordinates
la_combined = subset_dataset_by_coords(processed_dataset, 34.05, -118.25)

# Print dataset information
print(la_combined)
print(la_combined.dims)
print(la_combined.variables)

In [ ]:
import xarray as xr
import numpy as np
from dask import array as da


def process_dataset(dataset):
    """
    Apply degree days calculation and drop tmax and tmin variables.
    """
    # Ensure tmax and tmin are dask arrays
    tmax = dataset["tmax"]
    tmin = dataset["tmin"]

    # Calculate degree days using map_blocks
    degree_days = da.map_blocks(
        lambda tmax, tmin: single_sine_horizontal_cutoff(tmax, tmin),
        tmax,
        tmin,
        dtype=float,
    )

    # Create a new DataArray with the degree days
    degree_days_da = xr.DataArray(degree_days, dims=dataset.dims, coords=dataset.coords)

    # Add degree days to the dataset and drop tmax and tmin
    dataset = dataset.assign(degree_days=degree_days_da)
    dataset = dataset.drop_vars(["tmax", "tmin"])

    return dataset


# Process the dataset
processed_la = process_dataset(la_combined)

processed_la = processed_la.compute()
# Subset dataset by coordinates

# Print dataset information

In [ ]:
import xarray as xr
import numpy as np
from dask import array as da


def calculate_degree_days(tmax, tmin, base_temp=10):
    """
    Calculate degree days from tmax and tmin.
    Degree days = (tmax + tmin) / 2 - base_temp
    """
    mean_temp = (tmax + tmin) / 2
    degree_days = mean_temp - base_temp
    return degree_days


def process_dataset(dataset):
    """
    Apply degree days calculation and drop tmax and tmin variables.
    """
    # Directly access tmax and tmin
    tmax = dataset["tmax"]
    tmin = dataset["tmin"]

    # Calculate degree days
    degree_days = xr.apply_ufunc(
        calculate_degree_days,
        tmax,
        tmin,
        input_core_dims=[["time"], ["time"]],
        output_core_dims=[["time"]],
        vectorize=True,
        dask="allowed",  # Ensure compatibility with dask
    )

    # Create a new DataArray with the degree days
    degree_days_da = xr.DataArray(
        degree_days,
        dims=dataset.dims,  # Use dimensions from the original dataset
        coords=dataset.coords,  # Use coordinates from the original dataset
        name="degree_days",
    )

    # Add degree days to the dataset and drop tmax and tmin
    dataset = dataset.assign(degree_days=degree_days_da)
    dataset = dataset.drop_vars(["tmax", "tmin"])

    return dataset


processed_dataset = process_dataset(la_combined)

# Persist or compute if needed
processed_dataset = processed_dataset.persist()

In [ ]:
print(la_combined)

In [ ]:
import dask.array as da
import numpy as np

# Define the shape of the array
lat, lon = 10, 10
days_per_year = 365
years = 2
variables = 2  # tmean and tmax
total_days = days_per_year * years

# Create larger random data for tmean and tmax to ensure cumulative sum can reach 50
tmean_data = np.random.rand(total_days, lat, lon) * 10
tmax_data = np.random.rand(total_days, lat, lon) * 10

# Stack the data along a new axis to create a single array
data = np.stack([tmean_data, tmax_data], axis=0)  # Shape will be (variables, total_days, lat, lon)

# Convert the numpy array to a dask array
dask_array = da.from_array(data, chunks=(variables, days_per_year//4, lat//2, lon//2))  # Adjust chunks as needed

# Add one to every day for each variable
dask_array_plus_one = dask_array + 1

# Split the array into separate years
dask_array_years = [dask_array_plus_one[:, i*days_per_year:(i+1)*days_per_year, :, :] for i in range(years)]

# Calculate the cumulative sum along the time (days) axis for each year
cumsum_arrays = [da.cumsum(year, axis=1) for year in dask_array_years]

# Function to find the first day when the cumulative sum is >= 50
def find_day_where_cumsum_is_50(cumsum_slice):
    condition_met = (cumsum_slice >= 50)
    if condition_met.any():
        return condition_met.argmax(axis=0)
    else:
        # Return days_per_year if the condition is never met
        return days_per_year * np.ones_like(cumsum_slice[0], dtype=int)

# Apply the function along the days axis (axis=1) for each year's cumulative sum
days_reaching_50_each_year = [da.apply_along_axis(find_day_where_cumsum_is_50, 1, cumsum) for cumsum in cumsum_arrays]

# Stack the results to get an array of shape (years, variables, lat, lon)
days_reaching_50_result = da.stack(days_reaching_50_each_year)

# Compute the result
days_reaching_50_result_computed = days_reaching_50_result.compute()

# Verification: Check some values from the result
print("Shape of days_reaching_50_result_computed:", days_reaching_50_result_computed.shape)
print("Days reaching 50 for tmean (variable 0) in year 1:")
print(days_reaching_50_result_computed[0, 0])
print("\nDays reaching 50 for tmax (variable 1) in year 1:")
print(days_reaching_50_result_computed[0, 1])
print("\nDays reaching 50 for tmean (variable 0) in year 2:")
print(days_reaching_50_result_computed[1, 0])
print("\nDays reaching 50 for tmax (variable 1) in year 2:")
print(days_reaching_50_result_computed[1, 1])


In [ ]:
ds = nc.Dataset("data/test/derived/PRISM_mean_2000-2020.nc", "r", format="NETCDF4")

In [ ]:
# take mean of netcdf "data/test/PRISM/2020/PRISM_combo_20200101.nc"

# Open the NetCDF file
dataset = nc.Dataset("data/test/PRISM/2020/PRISM_combo_20200101.nc", "r")

In [ ]:
dataset = nc.Dataset("data/test/PRISM/2020/PRISM_combo_20200101.nc", "r")

In [ ]:
import concurrent.futures
import time


def compute(x):
    time.sleep(2)  # Simulate a time-consuming computation
    return x * x


def main():
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submit tasks to the executor
        futures = [executor.submit(compute, i) for i in range(5)]

        for future in concurrent.futures.as_completed(futures):
            result = future.result()  # This blocks until the result is available
            print(f"Result: {result}")


if __name__ == "__main__":
    main()

In [ ]:
import netCDF4 as nc
import numpy as np


def extract_values_from_ncdf(file_path, coord, variables):
    """
    Extract values at a given coordinate from specified variables in a NetCDF file.

    Parameters:
    - file_path (str): Path to the NetCDF file.
    - coord (tuple): Tuple of (latitude, longitude) for the target coordinate.
    - variables (list): List of variable names to extract.

    Returns:
    - dict: A dictionary with variable names as keys and extracted values as values.
    """
    target_lat, target_lon = coord

    with nc.Dataset(file_path, "r") as dataset:
        # Get the latitude and longitude variables
        lat = dataset.variables["latitude"][:]
        lon = dataset.variables["longitude"][:]

        # Find the nearest indices for the given coordinates
        lat_idx = np.abs(lat - target_lat).argmin()
        lon_idx = np.abs(lon - target_lon).argmin()

        # Extract values for each variable
        values = tuple(
            dataset.variables[var][0, lat_idx, lon_idx].data.item() for var in variables
        )
        # note to potentially clip NCDF values
    return values


# Example usage:
file_path = "data/test/PRISM/2020/PRISM_combo_20200101.nc"
coord = (34.05, -118.25)
variables = ["tmin", "tmax"]
values = extract_values_from_ncdf(file_path, coord, variables)
print(values)

In [ ]:
import concurrent.futures
import netCDF4 as nc
import numpy as np


def extract_values(file_path, coord, variables):
    target_lat, target_lon = coord

    with nc.Dataset(file_path, "r") as dataset:
        lat = dataset.variables["latitude"][:]
        lon = dataset.variables["longitude"][:]

        lat_idx = np.abs(lat - target_lat).argmin()
        lon_idx = np.abs(lon - target_lon).argmin()

        values = tuple(
            dataset.variables[var][0, lat_idx, lon_idx].data.item() for var in variables
        )

    return values


def process_day(file_path, coord, variables, LTT, UTT):
    tmin, tmax = extract_values(file_path, coord, variables)
    return single_sine_horizontal_cutoff(tmin, tmax, LTT, UTT)


def process_days_concurrently(file_path, coord, variables, LTT, UTT, dates):
    results = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for date in dates:
            # Submit the process_day task to the executor
            future = executor.submit(process_day, file_path, coord, variables, LTT, UTT)
            futures.append(future)

        # Collect results as they complete
        for future in concurrent.futures.as_completed(futures):
            results.append(future.result())

    return results


# Example usage
file_path = "your_file.nc"
coord = (34.05, -118.25)
variables = ["tmin", "tmax"]
LTT = 10
UTT = 20
dates = ["2024-07-01", "2024-07-02", "2024-07-03"]

results = process_days_concurrently(file_path, coord, variables, LTT, UTT, dates)
print(results)

In [ ]:
import netCDF4 as nc
import numpy as np


def get_file_path_PRISM(year, month, day, base_dir):
    
    
    file_path = f"{base_dir}/{year}/PRISM_combo_{year}{month}{day}.nc"
    return file_path


def extract_values(file_path, coord, variables):
    target_lat, target_lon = coord

    with nc.Dataset(file_path, "r") as dataset:
        lat = dataset.variables["latitude"][:]
        lon = dataset.variables["longitude"][:]

        lat_idx = np.abs(lat - target_lat).argmin()
        lon_idx = np.abs(lon - target_lon).argmin()

        values = tuple(
            dataset.variables[var][0, lat_idx, lon_idx].data.item() for var in variables
        )

    return values


def process_days(start_date, coord, variables, LTT, UTT, target_GDD_value):
    accumulated_GDD_result = 0
    
    data_date = start_date
    while accumulated_GDD_result < target_GDD_value:
        date_path = get_file_path_PRISM(data_date.year, data_date.month, data_date.day, "data/test/PRISM/")

        tmin, tmax = extract_values(file_path, coord, variables)
        daily_result = single_sine_horizontal_cutoff(tmin, tmax, LTT, UTT)
        accumulated_GDD_result += daily_result
        

        
    return accumulated_result, total_days_processed

In [ ]:
get_file_paths(2020, "01", "01", "data/test/PRISM")

In [ ]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Open the NetCDF file

# Get the latitude and longitude variables
lat = dataset.variables["latitude"][:]
lon = dataset.variables["longitude"][:]

# Get the data variable (e.g., temperature)
data = dataset.variables["tmax"][:]

# Your target coordinate
target_lat = 34.05  # example latitude
target_lon = -118.25  # example longitude

# Find the nearest indices for the given coordinates
lat_idx = np.abs(lat - target_lat).argmin()
lon_idx = np.abs(lon - target_lon).argmin()

# Plot the data
plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)

# Plot the data as a contourf plot
plt.contourf(lon, lat, data[0, :, :], transform=ccrs.PlateCarree(), cmap="coolwarm")
plt.colorbar(label="Temperature")

# Plot the target point
plt.plot(target_lon, target_lat, "bo", markersize=10, transform=ccrs.PlateCarree())
plt.text(target_lon + 0.5, target_lat, "Target Point", transform=ccrs.PlateCarree())

# Add title and labels
plt.title("Temperature Map with Target Point")
plt.xlabel("Longitude")
plt.ylabel("Latitude")

# Show the plot
plt.show()

# Close the dataset
dataset.close()

In [ ]:
import dask.array as da
import numpy as np

# Define the shape of the array
lat, lon = 10, 10
days_per_year = 365
years = 2
variables = 2  # tmean and tmax
total_days = days_per_year * years

# Create larger random data for tmean and tmax to ensure cumulative sum can reach 50
tmean_data = np.random.rand(total_days, lat, lon) * 10
tmax_data = np.random.rand(total_days, lat, lon) * 10

# Stack the data along a new axis to create a single array
data = np.stack(
    [tmean_data, tmax_data], axis=0
)  # Shape will be (variables, total_days, lat, lon)

# Convert the numpy array to a dask array
dask_array = da.from_array(
    data, chunks=(variables, days_per_year // 4, lat // 2, lon // 2)
)  # Adjust chunks as needed

# Add one to every day for each variable
dask_array_plus_one = dask_array + 1

# Split the array into separate years
dask_array_years = [
    dask_array_plus_one[:, i * days_per_year : (i + 1) * days_per_year, :, :]
    for i in range(years)
]

# Calculate the cumulative sum along the time (days) axis for each year
cumsum_arrays = [da.cumsum(year, axis=1) for year in dask_array_years]


# Function to find the first day when the cumulative sum is >= 50
def find_day_where_cumsum_is_50(cumsum_slice):
    condition_met = cumsum_slice >= 50
    if condition_met.any():
        return condition_met.argmax(axis=0)
    else:
        # Return days_per_year if the condition is never met
        return days_per_year * np.ones_like(cumsum_slice[0], dtype=int)


# Apply the function along the days axis (axis=1) for each year's cumulative sum
days_reaching_50_each_year = [
    da.apply_along_axis(find_day_where_cumsum_is_50, 1, cumsum)
    for cumsum in cumsum_arrays
]

# Stack the results to get an array of shape (years, variables, lat, lon)
days_reaching_50_result = da.stack(days_reaching_50_each_year)

# Compute the result
days_reaching_50_result_computed = days_reaching_50_result.compute()

# Verification: Check some values from the result
print(
    "Shape of days_reaching_50_result_computed:", days_reaching_50_result_computed.shape
)
print("Days reaching 50 for tmean (variable 0) in year 1:")
print(days_reaching_50_result_computed[0, 0])
print("\nDays reaching 50 for tmax (variable 1) in year 1:")
print(days_reaching_50_result_computed[0, 1])
print("\nDays reaching 50 for tmean (variable 0) in year 2:")
print(days_reaching_50_result_computed[1, 0])
print("\nDays reaching 50 for tmax (variable 1) in year 2:")
print(days_reaching_50_result_computed[1, 1])

In [ ]:
import dask.array as da
import numpy as np

# Define the shape of the array
lat, lon = 10, 10
days_per_year = 365
years = 2
variables = 2  # tmean and tmax
total_days = days_per_year * years

# Create larger random data for tmean and tmax to ensure cumulative sum can reach 50
tmean_data = np.random.rand(total_days, lat, lon) * 10
tmax_data = np.random.rand(total_days, lat, lon) * 10

# Stack the data along a new axis to create a single array
data = np.stack(
    [tmean_data, tmax_data], axis=0
)  # Shape will be (variables, total_days, lat, lon)

# Convert the numpy array to a dask array
dask_array = da.from_array(
    data, chunks=(variables, days_per_year // 4, lat // 2, lon // 2)
)  # Adjust chunks as needed

# Add one to every day for each variable
dask_array_plus_one = dask_array + 1

# Split the array into separate years
dask_array_years = [
    dask_array_plus_one[:, i * days_per_year : (i + 1) * days_per_year, :, :]
    for i in range(years)
]

# Calculate the cumulative sum along the time (days) axis for each year
cumsum_arrays = [da.cumsum(year, axis=1) for year in dask_array_years]


# Function to find the first day when the cumulative sum is >= 50
def find_day_where_cumsum_is_50(cumsum_slice):
    condition_met = cumsum_slice >= 50
    if condition_met.any():
        return condition_met.argmax(axis=0)
    else:
        # Return days_per_year if the condition is never met
        return days_per_year * np.ones_like(cumsum_slice[0], dtype=int)


# Apply the function along the days axis (axis=1) for each year's cumulative sum
days_reaching_50_each_year = [
    da.apply_along_axis(find_day_where_cumsum_is_50, 1, cumsum)
    for cumsum in cumsum_arrays
]

# Stack the results to get an array of shape (years, variables, lat, lon)
days_reaching_50_result = da.stack(days_reaching_50_each_year)

# Compute the result
days_reaching_50_result_computed = days_reaching_50_result.compute()

# Verification: Check some values from the result
print(
    "Shape of days_reaching_50_result_computed:", days_reaching_50_result_computed.shape
)
print("Days reaching 50 for tmean (variable 0) in year 1:")
print(days_reaching_50_result_computed[0, 0])
print("\nDays reaching 50 for tmax (variable 1) in year 1:")
print(days_reaching_50_result_computed[0, 1])
print("\nDays reaching 50 for tmean (variable 0) in year 2:")
print(days_reaching_50_result_computed[1, 0])
print("\nDays reaching 50 for tmax (variable 1) in year 2:")
print(days_reaching_50_result_computed[1, 1])

In [ ]:
days_reaching_50_result.visualize()

In [ ]:
samp